In [1]:
import pandas as pd
import numpy as np

# Data Processin

In [ ]:
data = pd.read_csv('../BDBpoly_2209_ppargamma.tsv', usecols=[1, 11], low_memory=False, sep='\t')
data.dropna(inplace=True)
data = data.reset_index(drop=True)

In [ ]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from molvs.normalize import Normalizer, Normalization
from molvs.charge import Reionizer, Uncharger
from molvs import Standardizer

def molecule_preprocess(smi):

    mol = Chem.MolFromSmiles(smi)
    stand = Standardizer()
    mol = stand.standardize(mol)
    normalizer = Normalizer()
    new1 = normalizer.normalize(mol)
    remover = SaltRemover()
    new2 = remover.StripMol(new1)
    neutralize1 = Reionizer()
    new3 = neutralize1(new2)
    neutralize2 = Uncharger()
    new4 = neutralize2(new3)
    new_smiles = Chem.MolToSmiles(new4, kekuleSmiles=True)
    return new_smiles

In [ ]:
smiles_standard = []
for idx, smi in enumerate(total['Ligand SMILES']):
    try:
        smis = molecule_preprocess(smi)
        smiles_standard.append(smis)
    except:
        print(f'{idx},',data_del_sym.loc[idx, 'Ligand SMILES'])

In [ ]:
total['Standard_SMILES'] = smiles_standard
total.drop('Ligand SMILES', axis=1, inplace=True)

In [ ]:
total['EC50 (nM)'] = total['EC50 (nM)'].astype('float32')
f_data = total.drop_duplicates('Standard_SMILES', keep='first')
l_data = total.drop_duplicates('Standard_SMILES', keep='last')
f_l_data = pd.merge(f_data, l_data, how='outer', on='Standard_SMILES')

In [ ]:
for idx in f_l_data.index:
    f_l_data.loc[idx, 'EC50 (nM)'] = min(f_l_data.loc[idx, 'EC50 (nM)_x'], f_l_data.loc[idx, 'EC50 (nM)_y'])
f_l_data.drop(['EC50 (nM)_x', 'EC50 (nM)_y'], axis=1, inplace=True)
f_l_data['pEC50'] = f_l_data['EC50 (nM)'].apply(lambda x: round(-np.log10(x * 1e-9), 2))

In [ ]:
f_l_data['label'] = np.where(f_l_data['pEC50'].values > 6.0, 1, 0)
f_l_data['label'].value_counts()

# Molecular fingerprint generation

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
f_l_data = pd.read_csv('../process_PPARr_EC50.csv')
mols = [Chem.MolFromSmiles(smi) for smi in f_l_data['Standard_SMILES']]

ecfp6 = pd.DataFrame([AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048, useChirality=True).ToList() \
                     for mol in mols], columns=[f'mg{i}' for i in range(2048)])
ecfp6

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split, KFold
xtrain, xtest, ytrain, ytest = train_test_split(ecfp6, f_l_data['label'].values, test_size=0.3, stratify=f_l_data['label'].values, random_state=45)
xvalid, xtest, yvalid, ytest = train_test_split(xtest, ytest, train_size=(1/3), stratify=ytest, random_state=45)

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)

pred_va = rf.predict_proba(xvalid)
evaluateScorex(yvalid, pred_va)
pred_te = rf.predict_proba(xtest)
evaluateScorex(ytest, pred_te)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier()
et.fit(xtrain, ytrain)

pred_vae = et.predict_proba(xvalid)
pred_tee = et.predict_proba(xtest)
evaluateScorex(yvalid, pred_vae),evaluateScorex(ytest, pred_tee)

In [ ]:
import optuna
import sklearn
import xgboost as xgb
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(xtrain, ytrain, test_size=0.2)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 2**6)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.roc_auc_score(valid_y, preds)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

In [ ]:
params = study.best_params
extra = {
    "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
}
params.update(extra)

dtrain = xgb.DMatrix(xtrain, ytrain)
dvalid = xgb.DMatrix(xvalid, yvalid)
dtest = xgb.DMatrix(xtest)

xgbm = xgb.train(params, dtrain, num_boost_round=5000,evals=[(dtrain, "train"),(dvalid, "valid")], early_stopping_rounds=50)
pred_pro_val = xgbm.predict(dvalid, iteration_range=(0, xgbm.best_ntree_limit + 1))
pred_pro_te = xgbm.predict(dtest, iteration_range=(0, xgbm.best_ntree_limit + 1))
evaluateScorex(yvalid, pred_pro_val),evaluateScorex(ytest, pred_pro_te)

# Ten-fold cross-validation

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, matthews_corrcoef,balanced_accuracy_score
def evaluateScorex(y, pred_pro):
    try:
        auc = roc_auc_score(y, pred_pro[:, 1])
        pred = np.rint(pred_pro[:, 1])
    except:
        auc = roc_auc_score(y, pred_pro)
        pred = np.rint(pred_pro)
    bacc = balanced_accuracy_score(y, pred)
    mcc = matthews_corrcoef(y, pred)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    # kappa = cohen_kappa_score(y, pred)
    SE = tp / (tp + fn)
    SP = tn / (tn + fp)
    f1 = f1_score(y, pred)
    precision = precision_score(y, pred)
    recall = recall_score(y, pred)
    return auc, bacc, mcc, SE, SP, f1, precision, recall

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def cv_fold_model_rf(X, Y, xva, yva, xtest):
    X = X.reset_index(drop=True)
    kf = KFold(n_splits=10, shuffle=True)
    val_result = []
    pred1 = 0.0
    pred_test = 0.0
    pred_val_result = []
    models = []
    for k, (tridx, vaidx)in enumerate(kf.split(X, Y)):
        print(f'*******************{k + 1}*******************')
        
        xtrain, xvalid = X.loc[tridx,:], X.loc[vaidx,:]
        ytrain, yvalid = Y[tridx], Y[vaidx]
        
        rf = RandomForestClassifier()
        rf.fit(xtrain, ytrain,)
        pred_val_pro = rf.predict_proba(xvalid)
        auc, bacc, mcc, se, sp, f1, pre, rec = evaluateScorex(yvalid, pred_val_pro)
        print(f'Fold{k + 1 }，cross-validation scores, auc_score:{auc}, ba_score:{bacc}, \
        mcc_score:{mcc}, se/sp_score:{se}/ {sp},f1_score:{f1},precision_score:{pre},recall_score:{rec}')
        pred_val_result.append([auc, bacc, mcc, se, sp,f1,pre,rec])
        pred_test_pro = rf.predict_proba(xva)
        auc_test, bacc_test, mcc_test, se_test, sp_test, f1_test, pre_test, rec_test \
                                                                = evaluateScorex(yva, pred_test_pro)
        print(f'Fold{k + 1 }，the score of the validation set is, auc_score:{auc_test}, ba_score:{bacc_test}, mcc_score:{mcc_test}, se/sp_score:{se_test}/ {sp_test},f1_score:{f1_test},precision_score:{pre_test},recall_score:{rec_test}')
        pred1 += pred_test_pro
        val_result.append([auc_test, bacc_test, mcc_test, se_test, sp_test, f1_test,pre_test,rec_test])
        
        pred = rf.predict_proba(xtest)
        pred_test += pred
        models.append(rf)
    pred_mean = pred1 / 10
    mean_pred_eval = np.mean(pred_val_result, axis=0)
    mean_pred_eval1 = np.mean(val_result, axis=0)
    test_predmean = pred_test / 10
    print('Cross validation 10 fold average:', mean_pred_eval)
    print('Validation set 10 off average:', mean_pred_eval1)
    return pred_mean, models, test_predmean

In [ ]:
valid_pred_rf, model_rf, test_pred_rf = cv_fold_model_rf(xtrain, ytrain, xvalid, yvalid, xtest)

# Model Explanation

In [ ]:
import shap
shap.initjs()
explainxgb = shap.TreeExplainer(xgbm, xtest)
shapvaluexgb = explainxgb.shap_values(xtest)

In [ ]:
shap_v = (shapvaluerf[1] + shapvalueet[1] + shapvaluexgb[1]) / 3
shap_v

In [ ]:
shap.summary_plot(shapvaluexgb, xtest)